# Input

In [169]:
file_path = 'uml.txt'

with open(file_path, 'r') as file:

    uml = file.read()    # Read the file line by line
    # print(uml)


In [170]:
import re
# Regular expression to find comments
comment_pattern = re.compile(r'/\*.*?\*/', re.DOTALL)

# Find all comments in the text
comments = re.findall(comment_pattern, uml)

# Print the comments
for comment in comments:
    uml = uml.replace(comment, "")

In [171]:
final_uml = []
for line in uml.splitlines():
    if "//" in line:
        #print(line)
        line_no_comment = line.split("//")[0]
        #print(f"new line:{line_no_comment}")
        # line_no_comment += "\n"

        # final_uml = final_uml.replace(line, line_no_comment)
        
        final_uml.append(line_no_comment)
    else:
        final_uml.append(line)

uml = "\n".join(final_uml)


In [172]:
final_uml = uml
for line in uml.splitlines():
    if "enum" in line:
        # print(line)
        print(line.replace("{","(").replace("}", ")").replace("enum","").strip().replace(";", ""))
        final_uml = final_uml.replace(line, "")

uml = final_uml

RelationType(AND,OR,NOT)
Operation(CONTROLLER,SENSOR)


# Parse

## functions

In [173]:
import re

def extract_content_within_braces(input_string):
    # Define a regular expression pattern to match content within curly braces
    pattern = r'\{([^}]+)\}'

    # Use re.findall to find all matches in the input string
    matches = re.findall(pattern, input_string)

    
    # Return the list of matches
    return matches

# Example usage
input_string = "{bla bla} {example} {another example}"
result = extract_content_within_braces(input_string)

print("Original string:", input_string)
print("Content within braces:", result)

Original string: {bla bla} {example} {another example}
Content within braces: ['bla bla', 'example', 'another example']


In [174]:
def create_nested_list(matchs):
    nested_list = []
    for paragraph in matchs:
        true_line = []
        lines = paragraph.split('\n')
        for i in range(len(lines)):
            lines[i] = lines[i].strip()
            lines[i] = lines[i].split("//")[0].strip()
            if len(lines[i])>0 and not "position" in lines[i]:
                true_line.append(lines[i])

        
        nested_list.append(true_line)

    return nested_list


In [175]:
def find_class(uml):
    ans = []
    lines = uml.splitlines()

    for l in lines:
        if "class" in l:
            cls = l.replace("class","")
            cls = cls.strip()
            if not cls in ans:
                ans.append(cls.replace("{",""))
    return ans

In [176]:
def is_rel(line):
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA "
    rels = [contain, aso, uni_aso_l, uni_aso_r, isa]

    for rel in rels:
        if rel in line:
            return True
    return False

In [177]:
def process_rel(cls, line):
    line = line.split(";")[0]
    
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA "
    if "-<@>" in line:
        elements = line.split("-<@>")
        return elements[1] + " contain " + elements[0] + cls
    if "<@>-" in line:
        elements = line.split("<@>-")
        return elements[0] + " " + cls +  " contain " + elements[1]      

    if "--" in line:
        elements = line.split("--")
        return elements[0] +  " " +cls +  " associate " + elements[1]      

    if "->" in line:
        elements = line.split("->")
        return elements[0] +  " " +cls +  " associate " + elements[1] 
    
    if "<-" in line:
        elements = line.split("<-")
        return elements[0] +  " " +cls +  " associate " + elements[1]      
    
    if isa in line:
        return cls + line.replace("isA", " inherit ")
    
    return "cannot process:" + line

## inference

In [178]:
print(uml)

class SmartHome{
    1 <@>- * SmartRoom rooms;
    1 <@>- 1 Address address;
    1 <@>- 1 ActivityLog activityLog;
    1 <@>- * AutomationRule automationRules;
}

class Address{
    String city;
    String province;
    String streetNumber;
    String country;
    String postalCode;
    String streetName;
}

class SmartRoom{
    String roomName;
    1 -- * Device devices;
}

class Device{
    abstract;
    
    autounique identifier;
    boolean isActivated;
    * -> 1 DeviceType device;
}

class SensorDevice{
    
    isA Device;
    1 -- * SensorReading readings;
}

class SensorReading{
    Date timestamp;
    1 -> 1 MeasuredValue measuredValue;
}

class MeasuredValue{
    
    String nameOfValue;
    String value;
}

class ActuatorDevice{
    
    isA Device;
    1 -- * ControlCommand commands;
}

class ControlCommand{
    String commandName;
    Date timestamp;
    
    String status;
}

class DeviceType {
    
    String element;
    
    Operation operation;
}

class ActivityLog{

In [179]:
classes = find_class(uml)
print(classes)
len(classes)

['SmartHome', 'Address', 'SmartRoom', 'Device', 'SensorDevice', 'SensorReading', 'MeasuredValue', 'ActuatorDevice', 'ControlCommand', 'DeviceType ', 'ActivityLog', 'AutomationRule', 'AutomationTrigger', 'AutomationPrecondition']


14

In [180]:
result = extract_content_within_braces(uml)
result = create_nested_list(result)
print(result)
len(result)

[['1 <@>- * SmartRoom rooms;', '1 <@>- 1 Address address;', '1 <@>- 1 ActivityLog activityLog;', '1 <@>- * AutomationRule automationRules;'], ['String city;', 'String province;', 'String streetNumber;', 'String country;', 'String postalCode;', 'String streetName;'], ['String roomName;', '1 -- * Device devices;'], ['abstract;', 'autounique identifier;', 'boolean isActivated;', '* -> 1 DeviceType device;'], ['isA Device;', '1 -- * SensorReading readings;'], ['Date timestamp;', '1 -> 1 MeasuredValue measuredValue;'], ['String nameOfValue;', 'String value;'], ['isA Device;', '1 -- * ControlCommand commands;'], ['String commandName;', 'Date timestamp;', 'String status;'], ['String element;', 'Operation operation;'], ['1 -> * ControlCommand commandsLogged;', '1 -> * SensorReading sensorReadingsLogged;', '1 -> * AutomationTrigger automationTriggers;'], ['* -> 1 AutomationPrecondition precondition;', '* -> 1..* ActuatorDevice controllerToDo;', 'boolean isActivated;', '* -- 0..1 AutomationRule 

14

## DSL

In [181]:
for cls, atr in zip(classes, result):
    print(cls)
    cls_line = cls + "("
    has_atr = False
    for a in atr:
        print(a)
        
        if not is_rel(a):
            a = a.replace(";", ",")
            cls_line = cls_line + " " + a
            has_atr = True
        else:
            print("-"*10 + " rel below " + "-"*10)
            print(process_rel(cls, a).strip())
            print(" ")

    if has_atr:
        cls_line = cls_line[:-1]
    cls_line += ")"
    print(cls_line.replace("{",""))
    print()
    print( "=" * 40)
    print("\n")

SmartHome
1 <@>- * SmartRoom rooms;
---------- rel below ----------
1  SmartHome contain  * SmartRoom rooms
 
1 <@>- 1 Address address;
---------- rel below ----------
1  SmartHome contain  1 Address address
 
1 <@>- 1 ActivityLog activityLog;
---------- rel below ----------
1  SmartHome contain  1 ActivityLog activityLog
 
1 <@>- * AutomationRule automationRules;
---------- rel below ----------
1  SmartHome contain  * AutomationRule automationRules
 
SmartHome()



Address
String city;
String province;
String streetNumber;
String country;
String postalCode;
String streetName;
Address( String city, String province, String streetNumber, String country, String postalCode, String streetName)



SmartRoom
String roomName;
1 -- * Device devices;
---------- rel below ----------
1  SmartRoom associate  * Device devices
 
SmartRoom( String roomName)



Device
abstract;
autounique identifier;
boolean isActivated;
* -> 1 DeviceType device;
---------- rel below ----------
*  Device associate  1 D